In [1]:
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from scipy.ndimage import shift # 이미지 이동을 위한 라이브러리

print("모든 라이브러리 임포트 완료.")

# ==================================
# 1. 데이터 로드 및 분리
# ==================================
print("MNIST 데이터 로드 중...")
try:
    mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='liac-arff')
except Exception as e:
    mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')

X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)

X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]
print("데이터 로드 및 분리 완료 (훈련 60k, 테스트 10k)")


# ==================================
# 2. (Task 1) 원본(60k) 데이터로 최적 모델 찾기 (PCA 사용)
# ==================================
# 2-1. 스케일링
print("[Task 1] 원본(60k) 데이터 스케일링...")
scaler_original = StandardScaler() # 나중에 재사용하기 위해 변수명에 _original 추가
X_train_scaled = scaler_original.fit_transform(X_train)

# 2-2. PCA (차원 축소)
print("[Task 1] 원본(60k) 데이터 PCA 적용...")
pca_original = PCA(n_components=0.95) # 분산 95% 유지
X_train_pca = pca_original.fit_transform(X_train_scaled)
print(f"PCA로 특성 축소: {X_train_scaled.shape[1]} -> {pca_original.n_components_}")

# 2-3. 그리드 탐색
print("[Task 1] 원본(60k) 데이터로 그리드 탐색 시작...")
param_grid = {'n_neighbors': [3, 4, 5], 'weights': ['uniform', 'distance']}
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=3, n_jobs=-1, verbose=2)

grid_search.fit(X_train_pca, y_train)

best_params = grid_search.best_params_
best_model_original = grid_search.best_estimator_
print(f"[Task 1] 최적 파라미터 찾음: {best_params}")

# (비교 기준을 위해) 원본 모델의 테스트 정확도 미리 측정
X_test_scaled_original = scaler_original.transform(X_test)
X_test_pca_original = pca_original.transform(X_test_scaled_original)
y_pred_original = best_model_original.predict(X_test_pca_original)
accuracy_original = accuracy_score(y_test, y_pred_original)
print(f"[Task 1] 원본 모델 테스트 정확도: {accuracy_original:.4f}")

# ==================================
# 3. (Task 2) 데이터 증강 함수 정의
# ==================================
def shift_image(image, dx, dy):
    """
    이미지를 (dx, dy)만큼 이동시킵니다.
    image: 784-벡터
    dx: 수평 이동 (양수: 오른쪽, 음수: 왼쪽)
    dy: 수직 이동 (양수: 아래, 음수: 위)
    """
    image = image.reshape(28, 28)
    # cval=0: 빈 공간을 0(검은색)으로 채움
    shifted_image = shift(image, [dy, dx], cval=0, mode='constant')
    return shifted_image.flatten()

print("\n[Task 2] 데이터 증강 시작...")

# ==================================
# 4. (Task 2) 훈련 세트 증강 (300,000개 생성)
# ==================================
X_train_augmented = []
y_train_augmented = []

# (시간이 1-2분 정도 걸릴 수 있습니다)
for image, label in zip(X_train, y_train):
    # 1. 원본 추가
    X_train_augmented.append(image)
    y_train_augmented.append(label)

    # 2. 네 방향 이동본 추가
    X_train_augmented.append(shift_image(image, 1, 0))  # 오른쪽
    y_train_augmented.append(label)
    X_train_augmented.append(shift_image(image, -1, 0)) # 왼쪽
    y_train_augmented.append(label)
    X_train_augmented.append(shift_image(image, 0, 1))  # 아래
    y_train_augmented.append(label)
    X_train_augmented.append(shift_image(image, 0, -1)) # 위
    y_train_augmented.append(label)

# 리스트를 NumPy 배열로 변환
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

print(f"데이터 증강 완료. 새 훈련 세트 크기: {X_train_augmented.shape}")

# 중요: 증강된 데이터를 섞어줍니다. (현재 0,0,0,0,0, 1,1,1,1,1... 순서임)
print("증강된 훈련 세트 셔플 중...")
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]


# ==================================
# 5. (Task 2) 증강된 데이터에 전처리 적용
# ==================================
# 중요: 원본(60k) 데이터로 학습시킨 scaler_original 과 pca_original 을
# 그대로 .transform() 에 사용해야 합니다. (절대 .fit() 을 다시 하면 안 됨)

print("[Task 2] 증강된(300k) 데이터에 스케일링 적용 중...")
X_train_aug_scaled = scaler_original.transform(X_train_augmented)

print("[Task 2] 증강된(300k) 데이터에 PCA 적용 중...")
X_train_aug_pca = pca_original.transform(X_train_aug_scaled)
print("전처리 적용 완료.")


# ==================================
# 6. (Task 2) 확장된 데이터셋으로 "최선 모델" 훈련
# ==================================
print(f"\n[Task 2] '앞에서 찾은 최적 파라미터'({best_params})로 새 모델 훈련 시작...")
# n_jobs=-1: 모델이 예측할 때 사용할 코어 수 (훈련이 아님)
knn_clf_augmented = KNeighborsClassifier(**best_params, n_jobs=-1)

start_time = time.time()
# KNN의 'fit'은 데이터를 저장하는 것이라 금방 끝납니다.
knn_clf_augmented.fit(X_train_aug_pca, y_train_augmented)
end_time = time.time()
print(f"모델 'fit' 완료. (소요 시간: {end_time - start_time:.2f}초)")


# ==================================
# 7. (Task 2) 최종 성능 비교
# ==================================
print("\n[Task 2] 증강된 모델로 테스트 세트 예측 시작...")
print(" (!! 훈련 데이터 30만 개와의 거리 계산으로 1시간 이상 소요될 수 있습니다 !! )")
start_time = time.time()

# ⭐️ 이 .predict() 단계가 가장 오래 걸리는 부분입니다.
# (테스트 1만 개 * 훈련 30만 개 * PCA특성 개수 만큼의 계산 발생)
y_pred_augmented = knn_clf_augmented.predict(X_test_pca_original)

end_time = time.time()
print(f"예측 완료. (소요 시간: { (end_time - start_time) / 60 :.2f} 분)")

accuracy_augmented = accuracy_score(y_test, y_pred_augmented)

print("\n--- 🚀 최종 성능 비교 ---")
print(f"   원본 모델 정확도 (60k 훈련): {accuracy_original:.4f}")
print(f"증강된 모델 정확도 (300k 훈련): {accuracy_augmented:.4f}")

if accuracy_augmented > accuracy_original:
    print("🎉 성능 향상! 데이터 증강이 효과가 있었습니다.")
else:
    print("😅 성능이 향상되지 않았거나 오히려 하락했습니다.")

모든 라이브러리 임포트 완료.
MNIST 데이터 로드 중...
데이터 로드 및 분리 완료 (훈련 60k, 테스트 10k)
[Task 1] 원본(60k) 데이터 스케일링...
[Task 1] 원본(60k) 데이터 PCA 적용...
PCA로 특성 축소: 784 -> 331
[Task 1] 원본(60k) 데이터로 그리드 탐색 시작...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
[Task 1] 최적 파라미터 찾음: {'n_neighbors': 4, 'weights': 'distance'}
[Task 1] 원본 모델 테스트 정확도: 0.9529

[Task 2] 데이터 증강 시작...
데이터 증강 완료. 새 훈련 세트 크기: (300000, 784)
증강된 훈련 세트 셔플 중...
[Task 2] 증강된(300k) 데이터에 스케일링 적용 중...
[Task 2] 증강된(300k) 데이터에 PCA 적용 중...
전처리 적용 완료.

[Task 2] '앞에서 찾은 최적 파라미터'({'n_neighbors': 4, 'weights': 'distance'})로 새 모델 훈련 시작...
모델 'fit' 완료. (소요 시간: 0.14초)

[Task 2] 증강된 모델로 테스트 세트 예측 시작...
 (!! 훈련 데이터 30만 개와의 거리 계산으로 1시간 이상 소요될 수 있습니다 !! )
예측 완료. (소요 시간: 1.58 분)

--- 🚀 최종 성능 비교 ---
   원본 모델 정확도 (60k 훈련): 0.9529
증강된 모델 정확도 (300k 훈련): 0.9627
🎉 성능 향상! 데이터 증강이 효과가 있었습니다.
